Offline phase

Imports, Parameters, and Object Points

In [2]:
import cv2
import numpy as np
import glob

# Setting the expected chessboard pattern size (number of inner corners)
pattern_size = (9, 6)
square_size = 21.7  # length of each square in mm

# Preparing the object points (3D points in the chessboard coordinate system)
objp = np.zeros((pattern_size[0]*pattern_size[1], 3), np.float32)
objp[:, :2] = np.mgrid[0:pattern_size[0], 0:pattern_size[1]].T.reshape(-1, 2)
objp *= square_size

# Lists to store all object points and image points
object_points_all = []   # 3D points in real-world space
image_points_all = []    # 2D points in the image plane

# For images that were processed automatically
object_points_auto = []
image_points_auto = []

training_files = glob.glob('training_images/*.jpg')

Helper functions for corner detection

In [3]:
# CHOICE 5: Enhance input for automatic detection
# Function that enhances the input image to improve automatic chessboard corner detection
def preprocess_image(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray_eq = cv2.equalizeHist(gray)
    gray_blur = cv2.medianBlur(gray_eq, 5)
    return gray_blur

# Function that allows automatic inner corner detection using OpenCV's chessboard detection functions
def detect_corners_automatically(img, pattern_size):
    preprocessed = preprocess_image(img)
    try:
        ret, corners = cv2.findChessboardCornersSB(preprocessed, pattern_size, None)
    except Exception:
        # Fall back to findChessboardCorners with flags if findChessboardCornersSB is unavailable
        flags = cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_NORMALIZE_IMAGE
        ret, corners = cv2.findChessboardCorners(preprocessed, pattern_size, flags)
    return ret, corners

# Function that provides an interactive manual annotation interface
# Choice 3: Improves the localization of the four corner points by providing zoomed-in feedback, undo, and confirmation
def get_manual_corners(img):
    clicked_points = []
    img_copy = img.copy()
    original_img = img.copy()  # used to redraw the image after undo

    # Function to update the display of the manual annotation window
    def update_display():
        nonlocal img_copy
        img_copy = original_img.copy()
        for pt in clicked_points:
            cv2.circle(img_copy, (int(pt[0]), int(pt[1])), 5, (0, 0, 255), -1)
            font = cv2.FONT_HERSHEY_SIMPLEX
            cv2.putText(img_copy, f"{int(pt[0])},{int(pt[1])}", (int(pt[0]), int(pt[1])),
                        font, 0.5, (0, 0, 255), 1)
        cv2.imshow("Manual Annotation", img_copy)

     # Function that displays a zoomed-in view to help the user click more precisely
    def zoom_click_refinement(img, point, zoom_factor=4, window_size=400):
        x, y = int(point[0]), int(point[1])
        h, w = img.shape[:2]
        half = window_size // 2
        x1 = max(x - half, 0)
        y1 = max(y - half, 0)
        x2 = min(x + half, w)
        y2 = min(y + half, h)
        roi = img[y1:y2, x1:x2]
        zoomed = cv2.resize(roi, (roi.shape[1]*zoom_factor, roi.shape[0]*zoom_factor))
        win_name = "Zoomed Refinement"
        cv2.namedWindow(win_name, cv2.WINDOW_NORMAL)
        cv2.imshow(win_name, zoomed)
        refined_point = None

        # Callback function for the zoomed-in refinement window
        def zoom_callback(event, zx, zy, flags, param):
            nonlocal refined_point
            if event == cv2.EVENT_LBUTTONDOWN:
                rx = zx / zoom_factor
                ry = zy / zoom_factor
                refined_point = (x1 + rx, y1 + ry)
                cv2.destroyWindow(win_name)

        cv2.setMouseCallback(win_name, zoom_callback)
        while refined_point is None:
            key = cv2.waitKey(1)
            if key == 27:  # Esc cancels refinement
                refined_point = point
                cv2.destroyWindow(win_name)
                break
        return refined_point
    
    #  Mouse callback function to capture clicks using zoomed-in refinement
    def click_event(event, x, y, _flags, _params):
        nonlocal clicked_points, img_copy
        if event == cv2.EVENT_LBUTTONDOWN:
            initial_point = (x, y)
            refined = zoom_click_refinement(img, initial_point)
            clicked_points.append(refined)
            update_display()
            print(f"Point selected: {refined}")

    cv2.namedWindow("Manual Annotation", cv2.WINDOW_NORMAL)
    cv2.resizeWindow("Manual Annotation", 1280, 720)
    cv2.imshow("Manual Annotation", img_copy)
    cv2.setMouseCallback("Manual Annotation", click_event)

    print("Please click on the 4 outer corners of the chessboard in the following order:")
    print("1. Top-Left")
    print("2. Top-Right")
    print("3. Bottom-Right")
    print("4. Bottom-Left")
    print("Press 'u' to undo the last click. After 4 clicks, press 'a' to accept or 'u' to undo the final click.")

    while True:
        cv2.imshow("Manual Annotation", img_copy)
        key = cv2.waitKey(1) & 0xFF
        if key == ord('u'):
            if clicked_points:
                clicked_points.pop()
                update_display()
                print("Undid the last click.")
        if len(clicked_points) == 4:
            print("4 points have been selected. Press 'a' to accept these points, or 'u' to undo the final click.")
            key_confirm = cv2.waitKey(0) & 0xFF
            if key_confirm == ord('a'):
                break
            elif key_confirm == ord('u'):
                if clicked_points:
                    clicked_points.pop()
                    update_display()
                    print("Undid the last click. Please click again.")
        if key == 27:  # Esc key
            print("Manual annotation canceled. Not enough points selected.")
            break

    cv2.destroyWindow("Manual Annotation")

    if len(clicked_points) == 4:
        return clicked_points
    else:
        return None
    
# Function that linearly interpolates all chessboard points from the given outer corners
def interpolate_with_homography(corners, grid_size):
    num_cols, num_rows = grid_size

    # Define the ideal source points in a rectified coordinate system for the inner corners
    src_points = np.array([
        [0, 0],                     # top-left of inner corners
        [num_cols - 1, 0],          # top-right of inner corners
        [num_cols - 1, num_rows - 1],  # bottom-right
        [0, num_rows - 1]           # bottom-left
    ], dtype=np.float32)
    dst_points = np.array(corners, dtype=np.float32)
    H = cv2.getPerspectiveTransform(src_points, dst_points)

    # Generate the full grid of points using a list comprehension and convert to a NumPy array
    grid_points = np.array([[j, i] for i in range(num_rows) for j in range(num_cols)], dtype=np.float32)
    
    # Reshape grid_points to (num_rows, num_cols, 2)
    grid_points = grid_points.reshape(num_rows, num_cols, 2)
            
    # Transform the full grid to image points using the homography
    full_points = cv2.perspectiveTransform(grid_points.reshape(-1, 1, 2), H)
    full_points = full_points.reshape(num_rows, num_cols, 2)
    
    # Extract only the inner points (exclude the outer rows and columns)
    inner_points = full_points[1:-1, 1:-1, :]
    return inner_points.reshape(-1, 2)

# Function that determines whether the chessboard is placed vertically or horizontally
def determine_grid_size(corners, horizontal_grid_size=11, vertical_grid_size=8):
    tl = np.array(corners[0], dtype=np.float32)
    tr = np.array(corners[1], dtype=np.float32)
    bl = np.array(corners[3], dtype=np.float32)
    width = np.linalg.norm(tr - tl)
    height = np.linalg.norm(bl - tl)
    if width >= height:
        return (horizontal_grid_size, vertical_grid_size)
    else:
        return (vertical_grid_size, horizontal_grid_size)

Process Each Training Image

In [4]:
for idx, fname in enumerate(training_files):
    print(f"\nProcessing image: {fname}")
    img_train = cv2.imread(fname)
    if img_train is None:
        print("Failed to load image.")
        continue
    # Attempt automatic corner detection
    ret, corners = detect_corners_automatically(img_train, pattern_size)
    
    if ret:
        print("Automatic corner detection succeeded.")
         # Refine corner positions for better accuracy
        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
        corners_refined = cv2.cornerSubPix(cv2.cvtColor(img_train, cv2.COLOR_BGR2GRAY), corners, (11, 11), (-1, -1), criteria)

        # Add the refined corners and corresponding object points
        object_points_all.append(objp)
        image_points_all.append(corners_refined)

        # Store these in the automatic lists
        object_points_auto.append(objp)
        image_points_auto.append(corners_refined)

        # Draw the detected corners
        img_auto = img_train.copy()
        cv2.drawChessboardCorners(img_auto, pattern_size, corners_refined, ret)
        cv2.namedWindow("Automatic Corners", cv2.WINDOW_NORMAL)
        cv2.imshow("Automatic Corners", img_auto)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    else:
        print("Automatic corner detection failed; invoking manual annotation.")
        manual_corners = get_manual_corners(img_train)
        if manual_corners is not None:
            grid_size_manual = determine_grid_size(manual_corners, horizontal_grid_size=11, vertical_grid_size=8)
            corners_manual_full = interpolate_with_homography(manual_corners, grid_size_manual)

            # Add the refined corners and corresponding object points
            object_points_all.append(objp)
            image_points_all.append(corners_manual_full)

            # Draw the detected corners
            img_with_points = img_train.copy()
            for pt in corners_manual_full:
                x, y = int(pt[0]), int(pt[1])
                cv2.circle(img_with_points, (x, y), 3, (255, 0, 0), -1)
            cv2.namedWindow("Interpolated Points", cv2.WINDOW_NORMAL)
            cv2.resizeWindow("Interpolated Points", 1280, 720)
            cv2.imshow("Interpolated Points", img_with_points)
            cv2.waitKey(0)
            cv2.destroyWindow("Interpolated Points")
        else:
            print("Skipping image since manual annotation is not available.")


Processing image: training_images\WIN_20250211_10_59_26_Pro.jpg
Automatic corner detection succeeded.

Processing image: training_images\WIN_20250211_10_59_58_Pro.jpg
Automatic corner detection succeeded.

Processing image: training_images\WIN_20250211_11_00_39_Pro.jpg
Automatic corner detection succeeded.

Processing image: training_images\WIN_20250211_11_12_01_Pro.jpg
Automatic corner detection succeeded.

Processing image: training_images\WIN_20250211_11_12_06_Pro.jpg
Automatic corner detection succeeded.

Processing image: training_images\WIN_20250211_11_12_12_Pro.jpg
Automatic corner detection succeeded.

Processing image: training_images\WIN_20250211_11_17_19_Pro.jpg
Automatic corner detection succeeded.

Processing image: training_images\WIN_20250212_12_24_40_Pro.jpg
Automatic corner detection succeeded.

Processing image: training_images\WIN_20250212_12_24_45_Pro.jpg
Automatic corner detection succeeded.

Processing image: training_images\WIN_20250212_12_24_50_Pro.jpg
Automati

Process the final test image

In [5]:
# Load the final test image
final_test_file = "final_test.jpg"
img_final = cv2.imread(final_test_file)
if img_final is None:
    print("Error: Could not load final test image.")
else:
    # Convert the image to grayscale
    gray_final = cv2.cvtColor(img_final, cv2.COLOR_BGR2GRAY)
    
    # Attempt automatic corner detection using the enhanced function
    ret, corners_final = detect_corners_automatically(img_final, pattern_size)
    
    print("Final Test Image - Detection flag (ret):", ret)
    
    if ret and corners_final is not None and len(corners_final) > 0:
        print("Automatic corner detection succeeded on the final test image.")
        # Refine the detected corners for better accuracy using the correctly obtained variable
        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
        corners_final_refined = cv2.cornerSubPix(gray_final, corners_final, (11, 11), (-1, -1), criteria)

        # Add the refined corners and corresponding object points (if needed for calibration)
        object_points_all.append(objp)
        image_points_all.append(corners_final_refined)
        object_points_auto.append(objp)
        image_points_auto.append(corners_final_refined)
        
        # Draw the detected (refined) corners on a copy of the final image
        img_final_drawn = img_final.copy()
        cv2.drawChessboardCorners(img_final_drawn, pattern_size, corners_final_refined, ret)
        
        # Display the final test image with detected corners
        cv2.namedWindow("Final Test Automatic Detection", cv2.WINDOW_NORMAL)
        cv2.imshow("Final Test Automatic Detection", img_final_drawn)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    else:
        print("Automatic corner detection failed on the final test image. Please check the image conditions.")


Final Test Image - Detection flag (ret): True
Automatic corner detection succeeded on the final test image.


Camera Calibration Runs

In [6]:
# Choice 4: Function that computes the reprojection error for each image,
# providing a quantitative measure (confidence) of how well the camera parameters
# have been estimated
def compute_reprojection_errors(object_points, image_points, rvecs, tvecs, cameraMatrix, distCoeffs):
    total_error = 0
    total_points = 0
    errors = []
    for i in range(len(object_points)):
        # Project the 3D object points into the image plane
        imgpoints2, _ = cv2.projectPoints(object_points[i], rvecs[i], tvecs[i], cameraMatrix, distCoeffs)
        # Ensure that the detected image points are in the same shape and type as imgpoints2
        imgpoints_detected = np.array(image_points[i], dtype=np.float32).reshape(-1, 1, 2)
        # Compute the L2 norm (reprojection error) for this image
        error = cv2.norm(imgpoints_detected, imgpoints2, cv2.NORM_L2) / len(imgpoints2)
        errors.append(error)
        total_error += error * len(imgpoints2)
        total_points += len(imgpoints2)
    mean_error = total_error / total_points
    std_dev = np.std(errors)
    return mean_error, std_dev, errors


In [7]:
# Determine image size
img_example = cv2.imread(training_files[0])
img_size = (img_example.shape[1], img_example.shape[0])
print("\nImage size (width x height):", img_size)

# Run 1: Use all training images
ret1, cameraMatrix1, distCoeffs1, rvecs1, tvecs1 = cv2.calibrateCamera(
    object_points_all, image_points_all, img_size, None, None)
print("\nRun 1 Calibration Results (All Images):")
print("Camera Matrix:\n", cameraMatrix1)
print("Distortion Coefficients:\n", distCoeffs1)

mean_error1, std_error1, errors1 = compute_reprojection_errors(object_points_all, image_points_all, rvecs1, tvecs1, cameraMatrix1, distCoeffs1)
print("Mean Reprojection Error: {:.3f} pixels".format(mean_error1))
print("Standard Deviation: {:.3f}".format(std_error1))

# Run 2: Use only 10 images with automatic corner detections
if len(object_points_auto) >= 10:
    objpoints_run2 = object_points_auto[:10]
    imgpoints_run2 = image_points_auto[:10]
else:
    print("Not enough automatic images for Run 2; using available automatic images.")
    objpoints_run2 = object_points_auto
    imgpoints_run2 = image_points_auto

ret2, cameraMatrix2, distCoeffs2, rvecs2, tvecs2 = cv2.calibrateCamera(
    objpoints_run2, imgpoints_run2, img_size, None, None)
print("\nRun 2 Calibration Results (10 Automatic Images):")
print("Camera Matrix:\n", cameraMatrix2)
print("Distortion Coefficients:\n", distCoeffs2)

mean_error2, std_error2, errors2 = compute_reprojection_errors(objpoints_run2, imgpoints_run2, rvecs2, tvecs2, cameraMatrix2, distCoeffs2)
print("Mean Reprojection Error: {:.3f} pixels".format(mean_error2))
print("Standard Deviation: {:.3f}".format(std_error2))

# Run 3: Use only 5 images from the automatic ones
if len(objpoints_run2) >= 5:
    objpoints_run3 = objpoints_run2[:5]
    imgpoints_run3 = imgpoints_run2[:5]
else:
    print("Not enough images for Run 3; using available images from Run 2.")
    objpoints_run3 = objpoints_run2
    imgpoints_run3 = imgpoints_run2

ret3, cameraMatrix3, distCoeffs3, rvecs3, tvecs3 = cv2.calibrateCamera(
    objpoints_run3, imgpoints_run3, img_size, None, None)
print("\nRun 3 Calibration Results (5 Automatic Images):")
print("Camera Matrix:\n", cameraMatrix3)
print("Distortion Coefficients:\n", distCoeffs3)

mean_error3, std_error3, errors3 = compute_reprojection_errors(objpoints_run3, imgpoints_run3, rvecs3, tvecs3, cameraMatrix3, distCoeffs3)
print("Mean Reprojection Error: {:.3f} pixels".format(mean_error3))
print("Standard Deviation: {:.3f}".format(std_error3))


Image size (width x height): (1920, 1080)

Run 1 Calibration Results (All Images):
Camera Matrix:
 [[1.33933599e+03 0.00000000e+00 9.74589683e+02]
 [0.00000000e+00 1.33945326e+03 5.33368459e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
Distortion Coefficients:
 [[ 0.02261817 -0.00403822  0.00264624 -0.00613085 -0.31281871]]
Mean Reprojection Error: 0.116 pixels
Standard Deviation: 0.111

Run 2 Calibration Results (10 Automatic Images):
Camera Matrix:
 [[1.31618355e+03 0.00000000e+00 9.89617723e+02]
 [0.00000000e+00 1.31722910e+03 5.52073960e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
Distortion Coefficients:
 [[-9.34707985e-02  1.04046248e+00  7.20873138e-03  1.44415191e-03
  -2.22529826e+00]]
Mean Reprojection Error: 0.079 pixels
Standard Deviation: 0.026

Run 3 Calibration Results (5 Automatic Images):
Camera Matrix:
 [[1.39304935e+03 0.00000000e+00 1.00025686e+03]
 [0.00000000e+00 1.39116147e+03 5.49802805e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
Dist

Online phase

In [18]:
# Function to calculate color based on position and orientation
def calculate_hsv_color(rvec, tvec, top_plane_normal=[0, 0, 1]):
    # Calculate distance to camera (for intensity/value) and convert tvec (in mm) to meters
    distance = np.linalg.norm(tvec) / 1000.0
    value = max(0, 255 * (1 - distance/4))  # Linear scaling 0-4m to 255-0
    
    # Calculate angle between camera and top plane normal (for saturation)
    R, _ = cv2.Rodrigues(rvec)
    transformed_normal = R.dot(top_plane_normal)
    angle = np.arccos(transformed_normal[2]) * 180/np.pi
    saturation = max(0, 255 * (1 - angle/45))
    
    # Calculate hue based on relative position
    # Using horizontal position (x-coordinate) for hue
    hue = (np.arctan2(tvec[0][0], tvec[2][0]) + np.pi) * 180/np.pi
    
    return np.uint8([[[hue, saturation, value]]])

# Function to draw cube and axes
def draw_cube_and_axes(img, rvec, tvec, camera_matrix, dist_coeffs, square_size):
    # Define 3D points for coordinate axes (length = 3 squares)
    axis_length = 3 * square_size
    axes = np.float32([[0,0,0], [axis_length,0,0], [0,axis_length,0], [0,0,-axis_length]])
    
    # Define cube points (2x2x2 squares)
    cube_size = 2 * square_size
    cube_points = np.float32([
        [0,0,0], [cube_size,0,0], [cube_size,cube_size,0], [0,cube_size,0],
        [0,0,-cube_size], [cube_size,0,-cube_size], 
        [cube_size,cube_size,-cube_size], [0,cube_size,-cube_size]
    ])
    
    # Project points
    imgpts_axes, _ = cv2.projectPoints(axes, rvec, tvec, camera_matrix, dist_coeffs)
    imgpts_cube, _ = cv2.projectPoints(cube_points, rvec, tvec, camera_matrix, dist_coeffs)
    
    # Draw axes
    origin = tuple(map(int, imgpts_axes[0].ravel()))
    img = cv2.line(img, origin, tuple(map(int, imgpts_axes[1].ravel())), (255,0,0), 3)  # X axis
    img = cv2.line(img, origin, tuple(map(int, imgpts_axes[2].ravel())), (0,255,0), 3)  # Y axis
    img = cv2.line(img, origin, tuple(map(int, imgpts_axes[3].ravel())), (0,0,255), 3)  # Z axis
    
    # Draw cube
    imgpts_cube = np.int32(imgpts_cube).reshape(-1,2)
    
    # Draw bottom face
    img = cv2.drawContours(img, [imgpts_cube[:4]], -1, (0,255,0), 3)
    
    # Draw top face
    img = cv2.drawContours(img, [imgpts_cube[4:]], -1, (0,255,0), 3)
    
    # Draw vertical edges
    for i in range(4):
        img = cv2.line(img, tuple(imgpts_cube[i]), tuple(imgpts_cube[i+4]), (0,255,0), 3)
    
    # Get color for top polygon based on position and orientation
    hsv_color = calculate_hsv_color(rvec, tvec)
    bgr_color = cv2.cvtColor(hsv_color, cv2.COLOR_HSV2BGR).squeeze()
    
    # Draw filled top polygon
    top_face = imgpts_cube[4:].reshape((-1,1,2))
    cv2.fillConvexPoly(img, top_face, bgr_color.tolist())
    
    return img


# Process test image with all three calibration results
test_img = cv2.imread("final_test.jpg")

# Attempt automatic corner detection using the enhanced function
ret, corners = detect_corners_automatically(test_img, pattern_size)

if ret:
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
    corners = cv2.cornerSubPix(cv2.cvtColor(test_img, cv2.COLOR_BGR2GRAY), corners, (11, 11), (-1, -1), criteria)
    
    # Process with each calibration result
    for run, (camera_matrix, dist_coeffs) in enumerate([
        (cameraMatrix1, distCoeffs1),
        (cameraMatrix2, distCoeffs2),
        (cameraMatrix3, distCoeffs3)
    ], 1):
        # Find pose
        _, rvec, tvec = cv2.solvePnP(objp, corners, camera_matrix, dist_coeffs)
        
        # Draw results
        img_result = test_img.copy()
        img_result = draw_cube_and_axes(img_result, rvec, tvec, camera_matrix, dist_coeffs, square_size)
        
        # Display result
        cv2.namedWindow(f"Run {run} Result", cv2.WINDOW_NORMAL)
        cv2.imshow(f"Run {run} Result", img_result)
        cv2.waitKey(0)
        
    cv2.destroyAllWindows()
else:
    print("Could not detect corners in test image automatically")